In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [3]:
num_epochs = 5
batch_size = 128
num_classes = 10
learning_rate = 0.001

In [4]:
## Train and test datasets


In [5]:

train_dataset = torchvision.datasets.MNIST(root='../../data', train=True, transform=transforms.ToTensor(), download=True)

In [6]:
test_dataset = torchvision.datasets.MNIST(root='../../data', train=False, transform=transforms.ToTensor(), download=True)

In [7]:
### Train and test dataloaders

In [8]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size,shuffle=True)

In [9]:
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [10]:
#nn.Linear?

In [11]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                      nn.Conv2d(in_channels=1, out_channels=16, kernel_size=5, stride=1, padding=2),
                      nn.BatchNorm2d(num_features=16),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2)
                      )
        self.layer2 = nn.Sequential(
                      nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1, padding=2),
                      nn.BatchNorm2d(num_features=32),
                      nn.ReLU(),
                      nn.MaxPool2d(kernel_size=2, stride=2)
                      )
        self.fc1 = nn.Linear(in_features=32*7*7, out_features=10)
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        return out

In [12]:
model = ConvNet()

In [13]:
[(name, tsor.shape) for name, tsor in model.named_parameters()]

[('layer1.0.weight', torch.Size([16, 1, 5, 5])),
 ('layer1.0.bias', torch.Size([16])),
 ('layer1.1.weight', torch.Size([16])),
 ('layer1.1.bias', torch.Size([16])),
 ('layer2.0.weight', torch.Size([32, 16, 5, 5])),
 ('layer2.0.bias', torch.Size([32])),
 ('layer2.1.weight', torch.Size([32])),
 ('layer2.1.bias', torch.Size([32])),
 ('fc1.weight', torch.Size([10, 1568])),
 ('fc1.bias', torch.Size([10]))]

In [14]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [15]:
total_step = len(train_loader)

In [17]:
for epoch in range(num_epochs):
    for batch_id, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (batch_id+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, batch_id+1, total_step, loss.item()))
        

Epoch [1/5], Step [100/469], Loss: 0.0274
Epoch [1/5], Step [200/469], Loss: 0.0216
Epoch [1/5], Step [300/469], Loss: 0.1050
Epoch [1/5], Step [400/469], Loss: 0.0196
Epoch [2/5], Step [100/469], Loss: 0.0102
Epoch [2/5], Step [200/469], Loss: 0.0135
Epoch [2/5], Step [300/469], Loss: 0.0061
Epoch [2/5], Step [400/469], Loss: 0.0235
Epoch [3/5], Step [100/469], Loss: 0.0036
Epoch [3/5], Step [200/469], Loss: 0.0458
Epoch [3/5], Step [300/469], Loss: 0.0089
Epoch [3/5], Step [400/469], Loss: 0.0172
Epoch [4/5], Step [100/469], Loss: 0.0248
Epoch [4/5], Step [200/469], Loss: 0.0041
Epoch [4/5], Step [300/469], Loss: 0.0278
Epoch [4/5], Step [400/469], Loss: 0.0058
Epoch [5/5], Step [100/469], Loss: 0.0103
Epoch [5/5], Step [200/469], Loss: 0.0306
Epoch [5/5], Step [300/469], Loss: 0.0055
Epoch [5/5], Step [400/469], Loss: 0.0069


#### Test the model

In [19]:
model.eval() # batchnorm uses moving mean/variance instead of mini-batch mean/variance

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc1): Linear(in_features=1568, out_features=10, bias=True)
)

In [22]:
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        
        _, predicted = torch.max(outputs.data, 1)
        
        total += len(labels)
        
        correct += (predicted == labels).sum().item()
        
    
    print(f'Test accuracy of the model on 10000 images {correct/total * 100}')
        
        

Accuracy 98.97


In [ ]:
import numpy as np
a = np.random.randn(2048,50000000)
b = np.random.randn(2048,1)

In [30]:
% timeit np.linalg.norm (a-b)

ValueError: operands could not be broadcast together with shapes (2048,50,0,0) (2048,1) 